In [1]:
import colorama
import warnings
import seaborn as sns
import datetime
import time

import mysql.connector as connection
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import display


BLACK = '\u001b[30m'
RED = '\u001b[31m'
GREEN = '\u001b[32m'
YELLOW = '\u001b[33m'
BLUE = '\u001b[34m'
MAGENTA = '\u001b[35m'
CYAN = '\u001b[36m'
WHITE = '\u001b[37m'
RESET = '\u001b[0m'

BOLD = '\u001b[1m'
UNDERLINE = '\u001b[4m'
REVERSE = '\u001b[7m'

# Variable 
userid = ''
name = ''
age = ''
occupy = ''
gender = ''

def getDF(sql):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        # Connect to MySQL
        conn = connection.connect(user='root', password='12345678Aa', host='localhost', database='dads4002_MissingData')
        
        # Get data 
        df = pd.read_sql(sql, conn)
        if len(df) > 0:
            df.head()
        # Always close connection
        else: 
            df = []
        conn.close()

        return df

def editDF(sql):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        # Connect to MySQL
        conn = connection.connect(user='root', password='12345678Aa', host='localhost', database='dads4002_MissingData')
        cursor = conn.cursor()

        try:
            cursor = conn.cursor()
            cursor.execute(sql)   # Syntax error in query
            conn.commit()
            affect = cursor.rowcount
        except connection.Error as err:
            affect = 0
            print("Something went wrong: {}".format(err))

        conn.close()
        return affect
    
def addLog(user,action):
    with open('log.txt',mode='a', encoding='utf-8') as log_file:
        package = {
            "timestamp": datetime.datetime.now().strftime("%d-%b-%Y (%H:%M:%S.%f)"),
            "user_id": user,
            "action": action
        }
        log_file.write(str(package)+"\n")

#Changing Color function
def colour_print(text: str, *effects: str) -> None:
    """
    Print `text` using the ANSI sequences to change colour, etc

    :param text: The text to print.
    :param effects: The effect we want.  One of the constants
        defined at the start of this module.
    """
    effect_string = "".join(effects)
    output_string = "{0}{1}{2}".format(effect_string, text, RESET)
    print(output_string, flush=True)

# Identify users
def login(_userid):
    userid = input("Please enter your UserID (or '0' for new user): ") if _userid == '' else _userid
    sql = f''' SELECT * FROM CUSTOMER WHERE USER_ID = '{userid}' '''
    df = getDF(sql)

    if userid == '0' or len(df) == 0:
        colour_print("Please register.", RED)
        registry()
    else:
        package = {
            'userid': df.iloc[0]["USER_ID"],
            'name' : df.iloc[0]["NAME"],
            'age' : df.iloc[0]["AGE"],
            'occupy' : df.iloc[0]["OCCUPY"],
            'gender' : df.iloc[0]["GENDER"],
        }
        # log
        addLog(df.iloc[0]["USER_ID"], "Login")

        
        colour_print(f"""Welcome, {df.iloc[0]["NAME"]}""", GREEN)
        colour_print(f"""User ID : {df.iloc[0]["USER_ID"]} (Please save this for later use!)""", GREEN)
        print("\n", flush=True)

        
        launch(mode,package)

def registry():
    while True:
        name = input("Enter your fullname (eg: John Doe): ")
        print("Name : ",name, flush=True)
        age = int(input("Enter your age (only integer): "))
        print("Age : ",age, flush=True)
        occupy = input("Enter your occupation: ")
        print("Occupation : ",occupy, flush=True)
        gender = input("Enter your gender (Male/Female): ")
        print("Gender : ",gender, flush=True)
        print("\n")
        break
    _name = name.split(' ') 
    userid = 'A20' + (_name[0]+_name[1])[:6] + str(age)
    sql = f'''  INSERT INTO CUSTOMER (USER_ID, NAME, AGE, OCCUPY, GENDER)
        VALUES ('{userid}', '{name}', '{age}', '{occupy}', '{gender}')'''
    affect = editDF(sql)
    
    if affect == 0:
        print("Something wrong. Please register again.", flush=True)
        registry()
    else:
        # log
        addLog(userid, "Register")
        login(userid)

#Main Menu
def mode():
    
    colour_print("(current) Main Menu", YELLOW)
    print("Please choose Mode (from 1 - 3)", flush=True)
    print("1 : Review", flush=True)
    print("2 : Analytic Reports", flush=True)
    print("3 : End", flush=True)
    print("\n", flush=True)

    
    mode = input("Please choose Mode (enter a number) : ")

    return mode

#For launching main menu
def launch(mode,user):
    mode = mode()
    if mode=='1':
        review_menu(user)
    elif mode=='2':
        analytic_reports(user)
    elif mode=='3':
        end()
    else:
        colour_print("\nInvalid input, please choose again", RED)
        rerun(user)

#Back to main menu
def rerun(user):
    launch(mode,user)

#Reiew mode (ฝากเติมหน่อยนะ)
def review_menu(user):
    
    colour_print("(current) Review Menu", YELLOW)
    print("Please choose search menu for your books (from 1 - 3)", flush=True)
    print("1 : Search by Book title", flush=True)
    print("2 : Search by Authors", flush=True)
    print("3 : Back to previous menu", flush=True)
    print("\n", flush=True)
    
    
    menu = input("Please choose search menu (enter a number) : ")

    
    if menu == '1':
        colour_print("(current) 1 : Search by Book title", YELLOW)
        title = input("Please enter keyword of book title : ")
        query = f""" SELECT * FROM BOOK_DETAIL WHERE title like '%{title}%' """
        df = getDF(query)
        
        if len(df) == 0:
            colour_print(f"{title} not found", RED)
            print("\n", flush=True)
            review_menu(user)
        else:
            display(df)
            print("1: Review ",df.iloc[0]["TITLE"], flush=True)
            print("2: Back to search", flush=True)
            print("\n", flush=True)

            
            sub_menu = input("Please choose search menu (enter a number) : ")

            if sub_menu == '1':
                colour_print(("(current) 1: Review ",df.iloc[0]["TITLE"]), YELLOW)
                print("\n", flush=True)
                review(df.iloc[0],user)   
            else:
                review_menu(user)

    elif menu == '2':
        colour_print("(current) 2 : Search by Authors", YELLOW)
        author = input("Please enter keyword of book's author : ")
        query = f""" SELECT * FROM BOOK_DETAIL WHERE authors like '%{author}%' """
        df = getDF(query)

        if len(df) == 0:
            colour_print(f"{author} not found", RED)
            print("\n", flush=True)
            review_menu(user)
        else:
            display(df)
            print("1: Review ",df.iloc[0]["TITLE"], flush=True)
            print("2: Back to search", flush=True)
            print("\n", flush=True)
            
            
            sub_menu = input("Please choose search menu (enter a number) : ")

            if sub_menu == '1' :
                colour_print(("(current) 1: Review ",df.iloc[0]["TITLE"]), YELLOW)
                print("\n", flush=True)
                review(df.iloc[0],user)   
            else:
                review_menu(user)
    else:
        rerun(user)

def review(df,user):
    score = int(input("Enter Review Score (1-5): "))
    if score < 0 or score >5:
        score = int(input("Enter Review Score : "))
    print("Review Score: ", score, flush=True)  

    txt = input("Enter Review Summary : ")
    print("Review Summary: ", txt, flush=True)  

    sql1 = f''' SELECT max(id_no)+1 ID_NO FROM book_rating '''
    query = getDF(sql1)
    id_no = query.iloc[0]["ID_NO"]

    sql2 = f''' INSERT INTO BOOK_RATING (ID_NO, TITLE, USER_ID, REVIEW_SCORE, REVIEW_TIME, REVIEW_TEXT) 
        VALUES ('{id_no}', '{df["TITLE"]}', '{user["userid"]}', '{score}', (SELECT DATE_FORMAT(now(), "%Y-%m-%d")),'{txt}') 
    '''
    
    affect = editDF(sql2)

    if affect == 0:
        colour_print("Review Unsuccessful!", RED)
        review(df)
        print("\n", flush=True)
    else:
        # log
        addLog(user["userid"], "Review")

        colour_print("Review Successful!", GREEN)
        review_menu(user)
        print("\n", flush=True)

#Analytic report mode
def analytic_reports(user):
    
    colour_print("(current) Analytic Reports", YELLOW)
    print("Please choose analytic report to show (from 1 - 4)", flush=True)
    print("1 : Distribution of Reviewer Age and Distribution of Book by Year", flush=True)
    print("2 : Average Rating of Book by Category", flush=True)
    print("3 : Rating probability of Authors to score rating > 4", flush=True)
    print("4 : Back to previous menu", flush=True)
    print("\n", flush=True)
    
    
    data = input("Please choose report (enter a number) : ")

    
    if data == '1':
        colour_print("(current) 1 : Distribution of Reviewer Age and Distribution of Book by Year", YELLOW)
        query = f""" SELECT Age FROM CUSTOMER """
        df = getDF(query)

        plt.figure(figsize=(8,6))
        sns.histplot(df["Age"].dropna(), bins=50, color="blue", kde=True)

        # Add labels and title
        plt.xlabel("Age")
        plt.xlim(right=120)
        plt.ylabel("Count")
        plt.title("Distribution of Reviewer Age")
        plt.show()

        query = f""" SELECT * FROM BOOK_DETAIL """
        book_df = getDF(query)

        year_counts = book_df.groupby("PUBLISHEDDATE")["TITLE"].count().sort_values(ascending=False)[:10]
        year_counts.plot(kind="bar")
        plt.xticks(rotation=90)
        plt.xlabel("Year")
        plt.ylabel("Number of books published")
        plt.title("Distribution of Book by Year")
        plt.show()
        
        
        max_year_count = year_counts.max()
        max_year = year_counts.idxmax()
        print(f"The year with the most published books was {max_year} with {max_year_count} books.", flush=True)
        print("\n", flush=True)

    elif data == '2':
        colour_print("(current) 2 : Average Rating of Book by Category", YELLOW)
        
        book_df = getDF(f'''select i.*,j.book
            from (select categories, MAX(x.score) as score
                    from (SELECT  a.title as book, b.categories, avg(REVIEW_SCORE) score
                    FROM book_rating a, book_detail b
                    where a.title = b.title
                    group by book) x
                group by categories) i
                INNER JOIN 
            (SELECT  a.title as book, b.categories, avg(REVIEW_SCORE) score
                    FROM book_rating a, book_detail b
                    where a.title = b.title
                    group by book) j
            ON i.categories = j.categories and i.score = j.score
            where i.categories != ''
            order by score desc
                ''')
        display(book_df)
        print("\n", flush=True)

    elif data == '3':
        colour_print("(current) 3 : Rating probability of Authors to score rating > 4", YELLOW)
        query = f""" SELECT a.user_id, a.review_score , b.title, b.authors
            FROM book_rating a,book_detail b
            where a.title = b.title
            and b.authors != ''
        """
        book_ratings_df = getDF(query)

        book_ratings_df.head()

        for x in book_ratings_df.sort_values(by='authors', ascending=True)["authors"].unique():
            author = x
            prob_author = book_ratings_df[(book_ratings_df["authors"] == author) & (book_ratings_df["review_score"] >= 4)]["authors"].count() / book_ratings_df[book_ratings_df["authors"] == author]["authors"].count()
            print(f"The probability of the author named {author}'s rating is {prob_author:.2f}", flush=True)
        print("\n", flush=True)

    elif data == '4':
        rerun(user)
    else:
        colour_print("\nInvalid input, please choose again", RED)
        analytic_reports(user)

    
    print("Please choose a menu (from 1 - 2)", flush=True)
    print("1 : Back to previous menu", flush=True)
    print("2 : Back to main menu", flush=True)
    print("\n", flush=True)
    
    
    menu = input("Please choose search menu (enter a number) : ")

    if menu == '1':
        analytic_reports(user)
    else:
        rerun(user)

#End program
def end():
    colour_print("--------------- Finished ---------------", GREEN)


login('')

Please enter your UserID (or '0' for new user): 
Please register.
Enter your fullname (eg: John Doe): Merlin Verginia
Name :  Merlin Verginia
Enter your age (only integer): 40
Age :  40
Enter your occupation: employee
Occupation :  employee
Enter your gender (Male/Female): Male
Gender :  Male


Welcome, Merlin Verginia
User ID : A20Merlin40 (Please save this for later use!)


(current) Main Menu
Please choose Mode (from 1 - 3)
1 : Review
2 : Analytic Reports
3 : End


Please choose Mode (enter a number) : 1
(current) Review Menu
Please choose search menu for your books (from 1 - 3)
1 : Search by Book title
2 : Search by Authors
3 : Back to previous menu


Please choose search menu (enter a number) : 2
(current) 2 : Search by Authors
Please enter keyword of book's author : nutt
nutt not found


(current) Review Menu
Please choose search menu for your books (from 1 - 3)
1 : Search by Book title
2 : Search by Authors
3 : Back to previous menu


Please choose search menu (enter a number) :